In [2]:
from __future__ import print_function
import argparse
import importlib

In [3]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
import mnist; importlib.reload(mnist)

<module 'mnist' from '/home/hminle/Github/weekly-ml-projects/variational-auto-encoder/mnist.py'>

In [4]:
# Define parser arguments
parser = {
    "batch_size": 128,
    "epochs": 10,
    "no_cuda": False,
    "seed": 1,
    "log_interval": 10,
}

In [5]:
# Parse arguments for  model
args = argparse.Namespace(**parser) # parse arguments
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [6]:
args.cuda

False

In [7]:
# Define seed for torch
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
# num_workers: use core of machine
# pin_memory is useful for GPU
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
# Download data from datasets on the Internet
# 
train_loader = torch.utils.data.DataLoader(
    mnist.MNIST('./data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    mnist.MNIST('./data', train=False, transform=transforms.ToTensor()),
batch_size=args.batch_size, shuffle=True, **kwargs)

## Define Pytorch Model

In [9]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        # pictures with 28x28 --> 784
        self.fc1 = nn.Linear(784, 400) #400 neural
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x)) # input --> fc1 --> relu --> h1
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if args.cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        # z --> fc3 --> relu --> fc4 --> sigmoid
        # Sigmoid will normalize data into 0 - 1 interval
        h3 = self.relu(self.fc3(z)) 
        return self.sigmoid(self.fc4(h3))
    
    # every model need to have a forward def --> how data flow
    def forward(self, x):
        # input = batchsize x 28 x28 --> batchsize x 784
        # flatten input and put it into encode, -1 is to keep the original batch size
        # mu = mean ...
        # logvar = log variational
        mu, logvar = self.encode(x.view(-1, 784)) 
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

## Init model


In [10]:
model = VAE()
# if we use cuda, model.cuda will load parameter into GPU for us
if args.cuda:
    model.cuda()

In [12]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False

In [13]:
def loss_function(recon_x, x, mu, logvar):
    # recon_x: output of decoder
    # x: input of encoder
    BCE = reconstruction_function(recon_x, x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # KLD = KL
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    # loss = BCE + KLD --> need to check with paper
    return BCE + KLD

In [15]:
model.parameters

<bound method Module.parameters of VAE (
  (fc1): Linear (784 -> 400)
  (fc21): Linear (400 -> 20)
  (fc22): Linear (400 -> 20)
  (fc3): Linear (20 -> 400)
  (fc4): Linear (400 -> 784)
  (relu): ReLU ()
  (sigmoid): Sigmoid ()
)>

## Define optimizer and train def

In [17]:
# If we use pretrained model, and need to update only fc4:
#optimizer = optim.Adam(model.fc4.parameters(), lr=1e-3)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [18]:
def train(epoch):
    # This has any effect only on modules such as Dropout or BatchNorm.
    model.train()
    
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        # Variable for back-prop, it wraps input + gradient 
        # of loss function with respect to input
        data = Variable(data)
        if args.cuda:
            data = data.cuda() # if cuda True, transfer data to GPU
        # create buffer for grad parameters --> must have for pytorch
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        
        # Because we init data = Variable --> loss and other parameters = Variable
        loss = loss_function(recon_batch, data, mu, logvar)
        
        # Auto Gradient, not update weights yet
        loss.backward()
        
        train_loss += loss.data[0]
        
        # Update weights here
        optimizer.step()
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


## Define test

In [21]:
def test(epoch):
    # Different from def train --> using model.eval
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        if args.cuda:
            data = data.cuda()
        
        # Init data with Variable(volatile=True) 
        # --> other parameters do not need to bring gradient
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

## Train model

In [22]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 548.967590
Train Epoch: 1 [1280/60000 (2%)]	Loss: 310.296082
Train Epoch: 1 [2560/60000 (4%)]	Loss: 235.784332
Train Epoch: 1 [3840/60000 (6%)]	Loss: 224.120712
Train Epoch: 1 [5120/60000 (9%)]	Loss: 208.667755
Train Epoch: 1 [6400/60000 (11%)]	Loss: 207.585968
Train Epoch: 1 [7680/60000 (13%)]	Loss: 208.356522
Train Epoch: 1 [8960/60000 (15%)]	Loss: 201.860794
Train Epoch: 1 [10240/60000 (17%)]	Loss: 192.739700
Train Epoch: 1 [11520/60000 (19%)]	Loss: 196.974609
Train Epoch: 1 [12800/60000 (21%)]	Loss: 192.592743
Train Epoch: 1 [14080/60000 (23%)]	Loss: 176.308838
Train Epoch: 1 [15360/60000 (26%)]	Loss: 181.238693
Train Epoch: 1 [16640/60000 (28%)]	Loss: 172.837067
Train Epoch: 1 [17920/60000 (30%)]	Loss: 171.217438
Train Epoch: 1 [19200/60000 (32%)]	Loss: 164.194183
Train Epoch: 1 [20480/60000 (34%)]	Loss: 159.862610
Train Epoch: 1 [21760/60000 (36%)]	Loss: 160.299576
Train Epoch: 1 [23040/60000 (38%)]	Loss: 157.998581
Train Epoch: 1 [24320/60000 